# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# output_data_file
output_data_file = "weather_df2.csv"

weather_data_df = pd.read_csv(output_data_file)

weather_data_df 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.15,-70.92,41.00,93.0,90.0,8.05,CL,1589669430
1,Hithadhoo,-0.60,73.08,84.81,77.0,79.0,20.54,MV,1589669430
2,Norman Wells,65.28,-126.83,30.20,54.0,75.0,18.34,CA,1589669431
3,Bluff,-46.60,168.33,50.05,83.0,57.0,8.99,NZ,1589669431
4,Rikitea,-23.12,-134.97,77.07,84.0,100.0,9.01,PF,1589669431
...,...,...,...,...,...,...,...,...,...
565,Zhireken,52.82,117.31,51.17,72.0,44.0,8.61,RU,1589669542
566,San Rafael,-34.62,-68.33,68.95,25.0,0.0,9.06,AR,1589669542
567,Le Port,-20.94,55.29,71.60,88.0,75.0,1.12,RE,1589669542
568,Porto Belo,-27.16,-48.55,68.00,72.0,0.0,4.70,BR,1589669543


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key = g_key)

In [8]:
# locations
locations = weather_data_df[["Lat", "Lng"]]

# humidity
humidity = weather_data_df["Humidity"].astype(float)


# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating = False, max_intensity = 5,
                                   point_radius = 1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

#max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
#wind_speed = new_types_df["Wind Speed"] < 10
#cloudiness = new_types_df["Cloudiness"] == 0


#new_types_df[max_temp & wind_speed & cloudiness]

In [10]:
#new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
#columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"]
new_types_df = weather_data_df.copy()
max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] < 10

#new_types_df#[max_temp & wind_speed & cloudiness]
new_types_df = new_types_df[max_temp & wind_speed & cloudiness]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

# hotel_df["Coordinates"]=""
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Punta Arenas,CL,-53.15,-70.92,
1,Hithadhoo,MV,-0.60,73.08,
2,Norman Wells,CA,65.28,-126.83,
3,Bluff,NZ,-46.60,168.33,
4,Rikitea,PF,-23.12,-134.97,
...,...,...,...,...,...
565,Zhireken,RU,52.82,117.31,
566,San Rafael,AR,-34.62,-68.33,
567,Le Port,RE,-20.94,55.29,
568,Porto Belo,BR,-27.16,-48.55,


In [12]:
# coordinates_lat_lng = []

# target_city = hotel_df["City"]
# # target_city

# target_type = "hotel"

# hotel_df["Coordinates"] = hotel_df["Lat"] + "," + hotel_df["Lng"]

# radius = 5000

# params = {
#     "location:" : coordinates_lat_lng,
#     "types" : target_type,
#     "radius" : radius, 
#     "key" : g_key
# }

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# print("A HOTEL WITHIN A 5000 METER RADIUS")

# response = requests.get(base_url, params)
# # print(response.url)

# hotel_distance = response.json()
# print(json.dumps(hotel_distance, indent=4, sort_keys=True))


In [13]:
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params = params)

In [14]:
print(response.json())

{'html_attributions': [], 'next_page_token': 'CrQCIQEAADOIphb3KMu0h6N4-3uDxB29kLhDDWqY_umfDH__Hjpk3sn2_ByKPqtAagHHMtRQQdEOdACAbfrvF_TbWrz0uTJ5vW-6JAVAtUlit9_jQnacGsTCQE0QjGjsRM4TfYL4Eq8VC3OOzb05uz0AqBxjEuAB-ySxFxPancqgJ1vG3eKsGksRGJ8ZO65YF8WGM6zBLvU0rnVvfX-dKtVp6eqGiaAI20uPOQr0ygckbhZ_q24OF9CQW_Mi7OsP5bbjG1ACpjBVxXDXtB-xVylbfV3hVoxOnliakwEhBIjsgpAx8taChwzCFND6OV2Ke3wB7KeUtW0-vyKuQZbFqyHBtd6GWpOqn0OD4CqlaiHuPT3myO7WhCqp7Uj35P5cjAe3C-AGpBST_HVhGo8WjaY8X1h9XHESENiEeHF0_SrndEh_B1sUpvUaFMIxREjkgX12DgS1AcP1Pb2bVsU5', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -53.1648732, 'lng': -70.9087551}, 'viewport': {'northeast': {'lat': -53.16345017010728, 'lng': -70.90759852010727}, 'southwest': {'lat': -53.16614982989272, 'lng': -70.91029817989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'id': '5b9e3b0667cb1a1a0947cabbaad5d4df46ecec8a', 'name': 'Hotel Isla Rey Jorge', 'photos': [{'height': 1536, 'html_attributions': 

In [15]:
hotel_df.iloc[0][0]

'Punta Arenas'

In [16]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#       hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [17]:
hotel_name

['Hotel Isla Rey Jorge',
 'Pebbles Inn',
 'Heritage Hotel',
 'Foveaux Hotel',
 "Pension Maro'i",
 'Hotel Xaguate',
 'The Halyards Hotel',
 nan,
 'Chautauqua Harbor Hotel',
 'The Edgewater Resort and Spa',
 'Les Embruns Du Baril',
 'Blue Bay Lodge',
 nan,
 'Hilton Garden Inn Kauai Wailua Bay',
 '28 on Hoop',
 'Bayhaven Lodge',
 'Eastern & Oriental Hotel, Penang',
 'Aspen Suites Hotel Sitka',
 'Tanna Ikamir Bungalows and Accommodation',
 nan,
 'Paradise First Hotel',
 'Xinhua Jianguo Hotel',
 nan,
 'Graciosa Resort - Biosphere Island Hotel',
 'Blue Heaven Island',
 'Hilton Cairns',
 nan,
 'Hotel Dioba Gite',
 'Los Cauquenes Resort & Spa',
 'Naujaaraaluit – Clyde River Hotel',
 "Maria's Hotel Nicte-Ha and Suites",
 'Pousada do Amparo',
 'The Marine Hermanus',
 "Ellie's Villa",
 'Hôtel Telwa Bungalow',
 'Hotel Aurassi',
 'The Vic Boutique Hotel',
 'Blue Ridge Beach Resort',
 'Hotel Arctic',
 'Mercure Bengkulu',
 'Katsuura Hotel Mikazuki',
 'The Sebel Busselton',
 'Isles Sunset Lodge',
 'Ho

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Heatmap & display
heatmap = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Marker_Locations & display
marker_locations = [
    (-40.0, -59.9),
    (-23.0, -65.2),
    (40.5, 45.5),
    (55.6, 60.6)
]

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))